In [1]:
from mqt.bench import get_benchmark
from mqt.bench import CompilerSettings, QiskitSettings, TKETSettings, get_benchmark
from qiskit import *
import numpy as np

In [2]:
min_qubit = 3 # maximum number of qubit
max_qubit = 8 # minimum number of qubit
gaps = 1

In [3]:
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
circuits= []
secret_keys=[]
import random
def build_bv_circuit(n, secret_string : str = None):
#Building a Bernstein–Vazirani circuit with n input qubits. If secret_string is not given, generate a random one.
    if secret_string is None:
        secret_string = ''.join(random.choice("01") for _ in range(n)) 
    qc = QuantumCircuit(n+1, n) # Create circuit with n input qubits + 1 ancilla
    qc.x(n)               # put ancilla in |1>
    qc.h(range(n+1))      # apply H to all qubits
    # Oracle for s
    for i, bit in enumerate(secret_string):
        if bit == "1":
            qc.cx(i, n)
    qc.h(range(n))
    qc.measure(range(n), range(n))    
    return qc, secret_string

for n in range(min_qubit, max_qubit, gaps):  # step = 1 by default
    qc, secret = build_bv_circuit(n)
    circuits.append((n,qc))
    secret_keys.append((n,secret))
    print(f"\n--- Bernstein–Vazirani with n={n}, secret={secret} ---")
    #print(qc.draw())
    print("=================================================================================================")
print("circtuit dictionary",circuits)
print("secret-keys Qubit wise",secret_keys)


--- Bernstein–Vazirani with n=3, secret=101 ---

--- Bernstein–Vazirani with n=4, secret=0111 ---

--- Bernstein–Vazirani with n=5, secret=11110 ---

--- Bernstein–Vazirani with n=6, secret=010001 ---

--- Bernstein–Vazirani with n=7, secret=0000111 ---
circtuit dictionary [(3, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7765a9fdda90>), (4, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x776502a5df10>), (5, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x776502b9ab40>), (6, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x776502c1f9b0>), (7, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x776502aae3f0>)]
secret-keys Qubit wise [(3, '101'), (4, '0111'), (5, '11110'), (6, '010001'), (7, '0000111')]


In [4]:
from qiskit_aer import AerSimulator
from qiskit import *
from qiskit.providers.fake_provider import *
import numpy as np 
import os
from qiskit.primitives import StatevectorSampler
sampler = AerSimulator()
jobs=[]
for qubit_number,circuit_object in circuits: 
    job = sampler.run([circuit_object], shots=1000)
    results = job.result()
    counts= results.get_counts()
    jobs.append((qubit_number , counts))
print(jobs)

[(3, {'101': 1000}), (4, {'1110': 1000}), (5, {'01111': 1000}), (6, {'100010': 1000}), (7, {'1110000': 1000})]


In [24]:
# -------------------------------
# Hellinger Fidelity Implementation
# -------------------------------
def hellinger_fidelity_with_expected(p, q):
    """Compute Hellinger fidelity between measured distribution p (counts)
       and expected distribution q (counts).
       Both p and q should be dictionaries: {bitstring: counts}."""
    p_sum, q_sum = sum(p.values()), sum(q.values())
    if q_sum == 0:
        raise ValueError("Expected distribution invalid (all counts = 0)")
    p_norm = {k: v / p_sum for k, v in p.items()}
    q_norm = {k: v / q_sum for k, v in q.items()}
    print("-----------------")
    print(p,q)
    print("-----------------")
    overlap = 0.0
    for k in set(p_norm) | set(q_norm):  # union of keys
        overlap += np.sqrt(p_norm.get(k, 0) * q_norm.get(k, 0))
    return overlap**2
    
h_f=[]
for qubit,result in jobs:
    for secret_string,count in result.items():
        expected = {secret_string: 1000}   # or {secret_string: 1} works the same (it normalizes internally)
        calculated ={secret_string: count} 
        fidelity = hellinger_fidelity_with_expected(calculated, expected)
        h_f.append((qubit,fidelity))
        print("The number of Qubit is: ",qubit,"and the Secret String is:", secret_string,"and calucated Hellinger Fidelity: score is",fidelity)   

-----------------
{'101': 1000} {'101': 1000}
-----------------
The number of Qubit is:  3 and the Secret String is: 101 and calucated Hellinger Fidelity: score is 1.0
-----------------
{'1110': 1000} {'1110': 1000}
-----------------
The number of Qubit is:  4 and the Secret String is: 1110 and calucated Hellinger Fidelity: score is 1.0
-----------------
{'01111': 1000} {'01111': 1000}
-----------------
The number of Qubit is:  5 and the Secret String is: 01111 and calucated Hellinger Fidelity: score is 1.0
-----------------
{'100010': 1000} {'100010': 1000}
-----------------
The number of Qubit is:  6 and the Secret String is: 100010 and calucated Hellinger Fidelity: score is 1.0
-----------------
{'1110000': 1000} {'1110000': 1000}
-----------------
The number of Qubit is:  7 and the Secret String is: 1110000 and calucated Hellinger Fidelity: score is 1.0
